In [ ]:
from rdflib import Namespace
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF

We need a namespace (`BLDG`) to name the entities that will exist in our model. `NUM_VAVs` will be used to generate a number of VAVs for this example

In [ ]:
NUM_VAVs = 3
BLDG = Namespace("urn:my-building/")

Create an in-memory BuildingMOTIF instance and create a `my-building` model that we will populate by evaluating templates

In [ ]:
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("https://example.com")

Load in templates / classes from the Brick ontology (this may take 1-2 minutes):

In [ ]:
Library.load(ontology_graph="../libraries/brick/Brick.ttl")

We load the G36 template library and pull out the "vav-cooling-only" template, corresponding to the pointlist in Section 4.1 of Guideline 36

In [ ]:
lib = Library.load(directory="../libraries/ashrae/guideline36/")
system_specifications = lib.get_shape_collection().get_shapes_of_definition_type(BMOTIF.System_Specification)
print(f"lib contains {len(system_specifications)} system specifications: {system_specifications}")
vav_templ = lib.get_template_by_name("vav-cooling-only")

Now that we have the template, we do the following to create the VAV:

In [ ]:
# create a "name" for the VAV
vav_name = BLDG["vav-0"]
# evaluate the template with that name to make sure the VAV has that name
tmp = vav_templ.evaluate({"name": vav_name})
# check what parameters are left
print(tmp.parameters)

If we don't want to provide names for these yet, we can call `.fill()` to invent names for them. This is helpful for testing and prototyping

In [ ]:
bindings, vav_graph = tmp.fill(BLDG) # tell 'fill' to put the invented names in the BLDG namespace

The `vav_graph` object can now be added to our model:

In [ ]:
bldg.add_graph(vav_graph)

In [ ]:
# run for the other n-1 VAVs
for vav in range(1,NUM_VAVs):
    vav_name = BLDG[f"vav-{vav}"]
    _, vav_graph = vav_templ.evaluate({"name": vav_name}).fill(BLDG)
    bldg.add_graph(vav_graph)

Print out the resulting model

In [ ]:
print(bldg.graph.serialize(format="turtle"))

In [ ]:
# and save your work!
bm.session.commit()

Rewinding a little bit, you may have noticed that the generated graph above doesn't actually contain any metadata for the points and parts associated with the VAVs. This is because we didn't deal with dependencies at all. We can use `inline_dependencies()` to resolve other definitions required by the VAV template:

In [ ]:
# without resolving dependencies
_, g = vav_templ.fill(BLDG)
print(g.serialize())

In [ ]:
# with resolved dependencies
inlined = vav_templ.inline_dependencies()
_, g = inlined.fill(BLDG)
print(g.serialize())

In [ ]:
print(inlined.body.serialize(format='turtle'))

Run another example w/ chiller plant

In [ ]:
lib = Library.load(directory="../libraries/chiller-plant/")

In [ ]:
chiller = lib.get_template_by_name("chiller")
inlined = chiller.inline_dependencies()
print(inlined.body.serialize())
_, g = inlined.fill(BLDG)
print(g.serialize())